In [ ]:
#@title Copyright 2023 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_training_patches_computePixels.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_training_patches_computePixels.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Download training patches from Earth Engine

This demonstration shows how to get patches of imagery from Earth Engine for training ML models.  Specifically, use `computePixels` calls in parallel to quickly and efficiently write a TFRecord file.

## Imports

In [ ]:
from google.colab import auth
from google.api_core import retry
from IPython.display import Image
from matplotlib import pyplot as plt
from numpy.lib import recfunctions as rfn

import concurrent
import ee
import google
import io
import multiprocessing
import numpy as np
import requests
import tensorflow as tf

## Authentication and initialization

Use the Colab auth widget to get credentials, then use them to initialize Earth Engine.  During initialization, be sure to specify a project and Earth Engine's [high-volume endpoint](https://developers.google.com/earth-engine/cloud/highvolume), in order to make automated requests.

In [ ]:
# REPLACE WITH YOUR PROJECT!
PROJECT = 'your-project'

In [ ]:
auth.authenticate_user()

In [ ]:
credentials, _ = google.auth.default()
ee.Initialize(credentials, project=PROJECT, opt_url='https://earthengine-highvolume.googleapis.com')

## Define variables

In [ ]:
# REPLACE WITH YOUR BUCKET!
OUTPUT_FILE = 'gs://your-bucket/your-file.tfrecord.gz'

# Output resolution in meters.
SCALE = 10

# Pre-compute a geographic coordinate system.
proj = ee.Projection('EPSG:4326').atScale(SCALE).getInfo()

# Get scales in degrees out of the transform.
SCALE_X = proj['transform'][0]
SCALE_Y = -proj['transform'][4]

# Patch size in pixels.
PATCH_SIZE = 128

# Offset to the upper left corner.
OFFSET_X = -SCALE_X * PATCH_SIZE / 2
OFFSET_Y = -SCALE_Y * PATCH_SIZE / 2

# Request template.
REQUEST = {
      'fileFormat': 'NPY',
      'grid': {
          'dimensions': {
              'width': PATCH_SIZE,
              'height': PATCH_SIZE
          },
          'affineTransform': {
              'scaleX': SCALE_X,
              'shearX': 0,
              'shearY': 0,
              'scaleY': SCALE_Y,
          },
          'crsCode': proj['crs']
      }
  }

# Blue, green, red, NIR, AOT.
FEATURES = ['B2_median', 'B3_median', 'B4_median', 'B8_median', 'AOT_median']

# Bay area.
TEST_ROI = ee.Geometry.Rectangle(
    [-123.05832753906247, 37.03109527141115,
     -121.14121328124997, 38.24468432993584])
# San Francisco.
TEST_COORDS = [-122.43519674072265, 37.78010979412811]

TEST_DATE = ee.Date('2021-06-01')

# Number of samples per ROI, and per TFRecord file.
N = 64

# Specify the size and shape of patches expected by the model.
KERNEL_SHAPE = [PATCH_SIZE, PATCH_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

## Image retrieval functions

This section includes functions to compute a Sentinel-2 median composite and get a pacth of pixels from the composite, centered on the provided coordinates, as either a numpy array or a JPEG thumbnail (for visualization).  The functions that request patches are retriable and you can do that automatically by decorating the functions with [Retry](https://googleapis.dev/python/google-api-core/latest/retry.html).

In [ ]:
def get_s2_composite(roi, date):
  """Get a two-month Sentinel-2 median composite in the ROI."""
  start = date.advance(-1, 'month')
  end = date.advance(1, 'month')

  s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
  s2c = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
  s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

  s2c = s2c.filterBounds(roi).filterDate(start, end)
  s2Sr = s2Sr.filterDate(start, end).filterBounds(roi)

  def indexJoin(collectionA, collectionB, propertyName):
    joined = ee.ImageCollection(ee.Join.saveFirst(propertyName).apply(
        primary=collectionA,
        secondary=collectionB,
        condition=ee.Filter.equals(
            leftField='system:index',
            rightField='system:index'
        ))
    )
    return joined.map(lambda image : image.addBands(ee.Image(image.get(propertyName))))

  def maskImage(image):
    s2c = image.select('probability')
    return image.updateMask(s2c.lt(50))

  withCloudProbability = indexJoin(s2Sr, s2c, 'cloud_probability')
  masked = ee.ImageCollection(withCloudProbability.map(maskImage))
  return masked.reduce(ee.Reducer.median(), 8)


@retry.Retry()
def get_patch(coords, image):
  """Get a patch centered on the coordinates, as a numpy array."""
  request = dict(REQUEST)
  request['expression'] = image
  request['grid']['affineTransform']['translateX'] = coords[0] + OFFSET_X
  request['grid']['affineTransform']['translateY'] = coords[1] + OFFSET_Y
  return np.load(io.BytesIO(ee.data.computePixels(request)))


@retry.Retry()
def get_display_image(coords, image):
  """Helper to display a patch using notebook widgets."""
  point = ee.Geometry.Point(coords)
  region = point.buffer(64 * 10).bounds()
  url = image.getThumbURL({
      'region': region,
      'dimensions': '128x128',
      'format': 'jpg',
      'min': 0, 'max': 5000,
      'bands': ['B4_median', 'B3_median', 'B2_median']
  })

  r = requests.get(url, stream=True)
  if r.status_code != 200:
    raise google.api_core.exceptions.from_http_response(r)

  return r.content

In [ ]:
TEST_IMAGE = get_s2_composite(TEST_ROI, TEST_DATE)
image = get_display_image(TEST_COORDS, TEST_IMAGE)
Image(image)

In [ ]:
np_array = get_patch(TEST_COORDS, TEST_IMAGE)

In [ ]:
# This is a structured array.
print(np_array['B4_median'])

In [ ]:
display_array = rfn.structured_to_unstructured(np_array[['B4_median', 'B3_median', 'B2_median']])/5000
plt.imshow(display_array)
plt.show()

## Sampling functions

These are helper functions to get a random sample as a list of coordinates,  sample the composite (using `computePixels`) at each coordinate, serialize numpy arrays to `tf.Example` protos and write them into a file.  The sampling is handled in multiple threads using a `ThreadPoolExecutor`.

In [ ]:
def get_sample_coords(roi, n):
  """"Get a random sample of N points in the ROI."""
  points = ee.FeatureCollection.randomPoints(region=roi, points=n, maxError=1)
  return points.aggregate_array('.geo').getInfo()


def array_to_example(structured_array):
  """"Serialize a structured numpy array into a tf.Example proto."""
  feature = {}
  for f in FEATURES:
    feature[f] = tf.train.Feature(
        float_list = tf.train.FloatList(
            value = structured_array[f].flatten()))
  return tf.train.Example(
      features = tf.train.Features(feature = feature))


def write_dataset(image, sample_points, file_name):
  """"Write patches at the sample points into a TFRecord file."""
  future_to_point = {
    EXECUTOR.submit(get_patch, point['coordinates'], image): point for point in sample_points
  }

  # Optionally compress files.
  writer = tf.io.TFRecordWriter(file_name)

  for future in concurrent.futures.as_completed(future_to_point):
      point = future_to_point[future]
      try:
          np_array = future.result()
          example_proto = array_to_example(np_array)
          writer.write(example_proto.SerializeToString())
          writer.flush()
      except Exception as e:
          print(e)
          pass

  writer.close()

In [ ]:
EXECUTOR = concurrent.futures.ThreadPoolExecutor(max_workers=N)

In [ ]:
# These could come from anywhere.  Here is just a random sample.
sample_points = get_sample_coords(TEST_ROI, N)

# Sample patches from the image at each point.  Each sample is
# fetched in parallel using the ThreadPoolExecutor.
write_dataset(TEST_IMAGE, sample_points, OUTPUT_FILE)

## Check the written file

Load and inspect the written file by visualizing a few patches.

In [ ]:
def parse_tfrecord(example_proto):
  """Parse a serialized example."""
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)

dataset = tf.data.TFRecordDataset(OUTPUT_FILE)
dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)

In [ ]:
take_20 = dataset.take(20)

for data in take_20:
  rgb = np.stack([
      data['B4_median'].numpy(),
      data['B3_median'].numpy(),
      data['B2_median'].numpy()], 2) / 5000
  plt.imshow(rgb)
  plt.show()


## Where to go next

 - Learn about how to scale training data generation pipelines with Apache Beam in [this demo](https://github.com/GoogleCloudPlatform/python-docs-samples/tree/main/people-and-planet-ai/land-cover-classification).
 - Learn about training models on Vertex AI in [this doc](/earth-engine/guides/tf_examples#semantic-segmentation-with-an-fcnn-trained-and-hosted-on-vertex-ai).